# Folded Cascode OTA Sizing using gm/ID (SG13G2 HV Devices)

## Setup & Device Models

In [1]:
import numpy as np
from pygmid import Lookup as lk

nmos = lk('sg13_hv_nmos.mat')
pmos = lk('sg13_hv_pmos.mat')

## System Specifications & Required gm

In [2]:
vdd = 3.0
vcm = vdd/2
C_load = 1e-12      # Load capacitance
UGB = 5e6           # Target unity gain bandwidth

v_out = vcm         # Output common-mode voltage
v_inp = vcm         # Input common-mode voltage
v_tail = 0.6        # Source of input pair (tail node)

gm = 2*np.pi*UGB*C_load
print("Required gm = %.2f uS" % (gm*1e6))

Required gm = 31.42 uS


## gm/ID Design Choices

In [3]:
gmid_in   = 10   # Input pair
gmid_casc = 8   # Cascode devices
gmid_sink = 8    # Bottom current sinks
gmid_src  = 8   # PMOS top sources
gmid_tail = 8    # Tail current source

## Length Choices

In [4]:
L_in = 1.0
L_tail = 1.0
L_casc_n = 1.0
L_casc_p = 1.0
L_sink = 1.0
L_src = 1.0

## Derive node voltages from gmid choices

In [5]:
vds_sink     = 2 / gmid_sink          
vsb_sink     = 0.0
v_casc_n_src = vds_sink               #drain of M4/M5

# Top source drain = Vdsat_src
vds_src      = 2 / gmid_src          
vsb_src      = 0.0
v_fold       = vdd - vds_src          # folding node voltage = VDD - Vdsat_src

vds_in       = v_fold - v_tail        # input pair VDS
vsb_in       = v_tail

vds_casc_n   = v_out - v_casc_n_src   # NMOS cascode VDS
vsb_casc_n   = v_casc_n_src

vds_casc_p   = v_fold - v_out         # PMOS cascode VDS
vsb_casc_p   = vdd - v_fold

vds_tail     = v_tail
vsb_tail     = 0.0

print(f"v_casc_n_src = {v_casc_n_src:.3f} V")
print(f"v_fold       = {v_fold:.3f} V")
print(f"vds_in       = {vds_in:.3f} V")
print(f"vds_casc_n   = {vds_casc_n:.3f} V")
print(f"vds_casc_p   = {vds_casc_p:.3f} V")

v_casc_n_src = 0.250 V
v_fold       = 2.750 V
vds_in       = 2.150 V
vds_casc_n   = 1.250 V
vds_casc_p   = 1.250 V


## Current Allocation & Verification

In [6]:
Id_tail = 2*(gm / gmid_in)
Id_branch = Id_tail / 2

gm_in_actual = gmid_in * Id_branch
gm_casc_n_actual = gmid_casc * Id_branch
gm_casc_p_actual = gmid_casc * Id_branch
gm_sink_actual = gmid_sink * Id_branch
gm_src_actual = gmid_src * (2 * Id_branch)
gm_tail_actual = gmid_tail * Id_tail

print("Tail current = %.2f uA" % (Id_tail*1e6))
print("Branch current = %.2f uA" % (Id_branch*1e6))
print("--- ACHIEVED TRANSCONDUCTANCE (gm) ---")
print(f"Target Required gm  = {gm*1e6:.2f} µS")
print(f"Input Pair (XM1/2)  = {gm_in_actual*1e6:.2f} µS")
print(f"NMOS Cascode (XM6/7)= {gm_casc_n_actual*1e6:.2f} µS")
print(f"PMOS Cascode (XM8/9)= {gm_casc_p_actual*1e6:.2f} µS")
print(f"Bottom Sink (XM4/5) = {gm_sink_actual*1e6:.2f} µS")
print(f"Top Source (XM10/11)= {gm_src_actual*1e6:.2f} µS")

Tail current = 6.28 uA
Branch current = 3.14 uA
--- ACHIEVED TRANSCONDUCTANCE (gm) ---
Target Required gm  = 31.42 µS
Input Pair (XM1/2)  = 31.42 µS
NMOS Cascode (XM6/7)= 25.13 µS
PMOS Cascode (XM8/9)= 25.13 µS
Bottom Sink (XM4/5) = 25.13 µS
Top Source (XM10/11)= 50.27 µS


## Transistor Width Sizing

In [7]:
jd_in = nmos.lookup('ID_W', GM_ID=gmid_in, L=L_in, VDS=vds_in, VSB=vsb_in)
W_in = Id_branch / jd_in
W_in_round = max(round(W_in * 2) / 2, 0.5)

jd_tail = nmos.lookup('ID_W', GM_ID=gmid_tail, L=L_tail, VDS=vds_tail, VSB=vsb_tail)
W_tail = Id_tail / jd_tail
W_tail_round = max(round(W_tail * 2) / 2, 0.5)

jd_casc_n = nmos.lookup('ID_W', GM_ID=gmid_casc, L=L_casc_n, VDS=vds_casc_n, VSB=vsb_casc_n)
W_casc_n = Id_branch / jd_casc_n
W_casc_n_round = max(round(W_casc_n * 2) / 2, 0.5)

jd_casc_p = pmos.lookup('ID_W', GM_ID=gmid_casc, L=L_casc_p, VDS=vds_casc_p, VSB=vsb_casc_p)
W_casc_p = Id_branch / jd_casc_p
W_casc_p_round = max(round(W_casc_p * 2) / 2, 0.5)

jd_sink = nmos.lookup('ID_W', GM_ID=gmid_sink, L=L_sink, VDS=vds_sink, VSB=vsb_sink)
W_sink = Id_branch / jd_sink
W_sink_round = max(round(W_sink * 2) / 2, 0.5)

jd_src = pmos.lookup('ID_W', GM_ID=gmid_src, L=L_src, VDS=vds_src, VSB=vsb_src)
W_src = (2 * Id_branch) / jd_src  # Im10 = Im1 + Im8
W_src_round = max(round(W_src * 2) / 2, 0.5)

print(f"W_in = {W_in:.2f} um, rounded W = {W_in_round} um")
print(f"W_tail = {W_tail:.2f} um, rounded W = {W_tail_round} um")
print(f"W_casc_n = {W_casc_n:.2f} um, rounded W = {W_casc_n_round} um")
print(f"W_casc_p = {W_casc_p:.2f} um, rounded W = {W_casc_p_round} um")
print(f"W_sink = {W_sink:.2f} um, rounded W = {W_sink_round} um")
print(f"W_src = {W_src:.2f} um, rounded W = {W_src_round} um")


W_in = 1.92 um, rounded W = 2.0 um
W_tail = 2.31 um, rounded W = 2.5 um
W_casc_n = 1.10 um, rounded W = 1.0 um
W_casc_p = 3.80 um, rounded W = 4.0 um
W_sink = 1.30 um, rounded W = 1.5 um
W_src = 8.54 um, rounded W = 8.5 um


## Bias Voltage Generation

In [8]:
def find_vgs_for_gmid(device, target_gmid, L_val, vds_val=1.5, vsb_val=0.0):
    vgs_test   = np.linspace(0.1, 3.3, 1000)
    gmid_curve = device.lookup('GM_ID', VGS=vgs_test, L=L_val,
                                VDS=vds_val, VSB=vsb_val)
    vgs_test   = np.asarray(vgs_test).flatten()
    gmid_curve = np.asarray(gmid_curve).flatten()
    sort_idx   = np.argsort(gmid_curve)
    return float(np.interp(target_gmid, gmid_curve[sort_idx], vgs_test[sort_idx]))

vgs_in     = find_vgs_for_gmid(nmos, gmid_in,   L_in,     vds_in,     vsb_in)
vgs_sink   = find_vgs_for_gmid(nmos, gmid_sink,  L_sink,   vds_sink,   vsb_sink)
vgs_casc_n = find_vgs_for_gmid(nmos, gmid_casc,  L_casc_n, vds_casc_n, vsb_casc_n)

vgs_src    = abs(find_vgs_for_gmid(pmos, gmid_src,   L_src,    vds_src,    vsb_src))
vgs_casc_p = abs(find_vgs_for_gmid(pmos, gmid_casc,  L_casc_p, vds_casc_p, vsb_casc_p))

vbias_n_ref  = vgs_sink
vbias_n_casc = vds_sink  + vgs_casc_n      # = Vdsat_sink + VGS_casc_n
vbias_p_src  = vdd       - vgs_src
vbias_p_casc = vdd       - vds_src - vgs_casc_p   # = VDD - Vdsat_src - |VGS_casc_p|

print(f"vbias_n_ref  = {vbias_n_ref:.3f} V")
print(f"vbias_n_casc = {vbias_n_casc:.3f} V")
print(f"vbias_p_src  = {vbias_p_src:.3f} V")
print(f"vbias_p_casc = {vbias_p_casc:.3f} V")

vbias_n_ref  = 0.840 V
vbias_n_casc = 1.162 V
vbias_p_src  = 2.150 V
vbias_p_casc = 1.777 V


## Output Resistance & DC Gain Estimation

In [9]:
gm_gds_in = nmos.lookup('GM_GDS', GM_ID=gmid_in, L=L_in, VDS=vds_in, VSB=vsb_in)
gm_gds_sink = nmos.lookup('GM_GDS', GM_ID=gmid_sink, L=L_sink, VDS=vds_sink, VSB=vsb_sink)
gm_gds_casc_n = nmos.lookup('GM_GDS', GM_ID=gmid_casc, L=L_casc_n, VDS=vds_casc_n, VSB=vsb_casc_n)
gm_gds_src = pmos.lookup('GM_GDS', GM_ID=gmid_src, L=L_src, VDS=vds_src, VSB=vsb_src)
gm_gds_casc_p = pmos.lookup('GM_GDS', GM_ID=gmid_casc, L=L_casc_p, VDS=vds_casc_p, VSB=vsb_casc_p)

gds_in = gm_in_actual / gm_gds_in
gds_sink = gm_sink_actual / gm_gds_sink
gds_src = gm_src_actual / gm_gds_src
gds_casc_n = gm_casc_n_actual / gm_gds_casc_n
gds_casc_p = gm_casc_p_actual / gm_gds_casc_p

Rout_up = (gm_casc_p_actual / gds_casc_p) * (1 / gds_src)
Rout_down = (gm_casc_n_actual / gds_casc_n) * (1 / (gds_sink + gds_in))

R_out = 1 / (1/Rout_up + 1/Rout_down)
print(f"Calculated Output Resistance = {R_out/1e6:.2f} MOhm")

DC_Gain = gm_in_actual * R_out
print(f"Calculated DC Gain = {20*np.log10(DC_Gain):.1f} dB")

Calculated Output Resistance = 51.24 MOhm
Calculated DC Gain = 64.1 dB


## Frequency Response & Parasitics

In [10]:
gm_cdd_casc_n = nmos.lookup('GM_CDD', GM_ID=gmid_casc, L=L_casc_n, VDS=vds_casc_n, VSB=vsb_casc_n)
gm_cdd_casc_p = pmos.lookup('GM_CDD', GM_ID=gmid_casc, L=L_casc_p, VDS=vds_casc_p, VSB=vsb_casc_p)
gm_cdd_in = nmos.lookup('GM_CDD', GM_ID=gmid_in, L=L_in, VDS=vds_in, VSB=vsb_in)
gm_cdd_src = pmos.lookup('GM_CDD', GM_ID=gmid_src, L=L_src, VDS=vds_src, VSB=vsb_src)
gm_cgg_casc_p = pmos.lookup('GM_CGG', GM_ID=gmid_casc, L=L_casc_p, VDS=vds_casc_p, VSB=vsb_casc_p)

C_out_parasitic = abs(gm_casc_n_actual / gm_cdd_casc_n) + abs(gm_casc_p_actual / gm_cdd_casc_p)
C_load_total = C_load + C_out_parasitic

UGB_actual = gm_in_actual / (2 * np.pi * C_load_total)
print(f"True UGB (with parasitics) = {UGB_actual/1e6:.2f} MHz")

C_dd_in = abs(gm_in_actual / gm_cdd_in)
C_dd_src = abs(gm_src_actual / gm_cdd_src)
C_gs_casc_p = abs(gm_casc_p_actual / gm_cgg_casc_p) 

C_folding_node = C_dd_in + C_dd_src + C_gs_casc_p
print(f"Parasitic capacitance at folding node = {C_folding_node*1e15:.2f} fF")

f_nondominant = gm_casc_p_actual / (2 * np.pi * C_folding_node)
print(f"Non-dominant pole frequency = {f_nondominant/1e6:.2f} MHz")

PM = 180 - 90 - np.degrees(np.arctan(UGB_actual / f_nondominant))
print(f"Estimated Phase Margin = {PM:.1f} degrees")

True UGB (with parasitics) = 4.99 MHz
Parasitic capacitance at folding node = 18.91 fF
Non-dominant pole frequency = 211.53 MHz
Estimated Phase Margin = 88.6 degrees


## Headroom Check

In [11]:
Vds_stack = 2/gmid_casc + 2/gmid_casc + 2/gmid_sink
print("Estimated Vdsat stack =", Vds_stack, "V")

Estimated Vdsat stack = 0.75 V
